In [ ]:
import h3.api.numpy_int as h3int
from sklearn.cluster import DBSCAN
from pyproj import Transformer

In [ ]:
def get_parameters(df, cluster_res, per):
    eps_m = round(h3int.edge_length(cluster_res) * 1000)
    df[‘h3_cluster_index’] = df[‘H3_int_index_12’] \
    .apply(lambda x: h3int.h3_to_parent(x, cluster_res))
    
    df_cluster = df.groupby(‘h3_cluster_index’)[‘count_mmsi’].sum() \
    .reset_index() \
    .sort_values(‘count_mmsi’,ignore_index=True, ascending=False)
    df_cluster[‘cumsum’] = df_cluster[‘count_mmsi’].cumsum()
    tot = df_cluster.count_mmsi.sum()
    df_cluster[‘cumper’] = df_cluster[‘cumsum’] / tot
    
    min_points = df_cluster[df_cluster.cumper >= per].iloc[0].count_mmsi
    return eps_m, int(min_points)

In [ ]:
sdf_agg = aoi_naoi_sdf.filter(
    (F.col(‘area’) == ‘aoi’) & (F.col(‘stationary’) == ‘Y’)) \
.groupBy(‘H3_int_index_12’) \
.agg(F.count(‘mmsi’).alias(‘count_mmsi’))

In [ ]:
df = sdf_agg.toPandas()
eps_m, min_points = get_parameters(df, 
 cluster_res = 8,
 per = 0.90)
model = DBSCAN(eps=eps_m,min_samples=min_points)

In [ ]:
points = df[‘H3_int_index_12’].apply(lambda x: h3int.h3_to_geo(x))
df[‘latitude’], df[‘longitude’] = zip(*points)

transf = Transformer.from_crs(4326, epsg_to=3832, always_xy=True)
(df[‘long_t’], df[‘lat_t’]) = transf.transform(df[‘longitude’].values,
                                               df[‘latitude’].values)

In [ ]:
X = df[[‘long_t’, ‘lat_t’]].values
sample_weight = df[‘count_mmsi’].values
model_fit = model.fit(X, sample_weight=sample_weight)

In [1]:
spark.stop()